In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.sql import DataFrame
import pyspark.sql.functions as F

In [2]:
# get or create Spark session

app_name = "spark-ratings-sentiments"
spark = SparkSession.builder.appName(app_name).getOrCreate()

In [3]:
import boto3

secret_name = "ut/postgres/db"
region_name = "us-east-2"
access_key = "YOUR ACCESS HERE"
secret_key = "YOUR KEY HERE"

session = boto3.session.Session(aws_access_key_id=access_key, aws_secret_access_key=secret_key, region_name=region_name)
client = session.client('secretsmanager')
secret_value = client.get_secret_value(SecretId=secret_name)
# secret_value

In [4]:
import json
def get_connection(secret_value):
  return json.loads(secret_value['SecretString'])
# get_connection(secret_value)

In [5]:
connection = get_connection(secret_value)

# Postgres credentials
jdbcHostname = connection['host']
jdbcPort = connection['port']
jdbcDatabase = "postgres"
dialect = "postgresql"
jdbcUsername = connection['username']
jdbcPassword = connection['password']

jdbcUrl = f"jdbc:{dialect}://{jdbcHostname}:{jdbcPort}/{jdbcDatabase}"
connectionProperties = {
  "user" : jdbcUsername,
  "password" : jdbcPassword,
  "driver" : "org.postgresql.Driver" 
}
# for mysql driver = com.mysql.jdbc.Driver

In [6]:
# Read from review_ratings table

table = "review_ratings"

df = spark.read.jdbc(url=jdbcUrl, table=table, properties=connectionProperties)
df.printSchema()

root
-- label: string (nullable = true)
-- ratings: double (nullable = true)
-- review_length: integer (nullable = true)

In [7]:
from pyspark.ml import Pipeline
import sparknlp
sparknlp.start()

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.4 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [8]:
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline

pipeline = PretrainedPipeline("analyze_sentiment", lang="en")

analyze_sentiment download started this may take some time.
Approx size to download 4.9 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]

In [9]:
help(pipeline.transform)

Help on method transform in module sparknlp.pretrained:

transform(data) method of sparknlp.pretrained.PretrainedPipeline instance

In [10]:
df.printSchema()

root
-- label: string (nullable = true)
-- ratings: long (nullable = true)
-- review_length: integer (nullable = true)

In [11]:
annotations_df = pipeline.transform(df.withColumnRenamed("label", "text"))
annotations_df.show()

+--------------------+-------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text|ratings|review_length| document| sentence| token| checked| sentiment|
+--------------------+-------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+
This place is awe...| 5.0| 263|[[document, 0, 26...|[[document, 0, 21...|[[token, 0, 3, Th...|[[token, 0, 3, Th...|[[sentiment, 0, 2...|
We were overcharg...| 1.0| 170|[[document, 0, 16...|[[document, 0, 43...|[[token, 0, 1, We...|[[token, 0, 1, We...|[[sentiment, 0, 4...|
Loved finding thi...| 5.0| 581|[[document, 0, 58...|[[document, 0, 64...|[[token, 0, 4, Lo...|[[token, 0, 4, mo...|[[sentiment, 0, 6...|
1 check-in Yes!! ...| 5.0| 214|[[document, 0, 21...|[[document, 0, 15...|[[token, 0, 0, 1,...|[[token, 0, 0, 1,...|[[sentiment, 0, 1...|
I live right by t...| 4.0| 752|[[document, 0, 75...|[[document, 0, 48...|[[token, 0, 0, I,...|[[token, 0, 0, I,...|[[sentiment, 0, 4...|
I love this Epoch...| 4.0| 294|[[document, 0, 29...|[[document, 0, 87...|[[token, 0, 0, I,...|[[token, 0, 0, I,...|[[sentiment, 0, 8...|
My second visit a...| 1.0| 388|[[document, 0, 38...|[[document, 0, 88...|[[token, 0, 1, My...|[[token, 0, 1, My...|[[sentiment, 0, 8...|
Stopped in for an...| 3.0| 283|[[document, 0, 28...|[[document, 0, 60...|[[token, 0, 6, St...|[[token, 0, 6, St...|[[sentiment, 0, 6...|
What I had: - Dri...| 4.0| 256|[[document, 0, 25...|[[document, 0, 49...|[[token, 0, 3, Wh...|[[token, 0, 3, Wh...|[[sentiment, 0, 4...|
I had their Affag...| 5.0| 251|[[document, 0, 25...|[[document, 0, 20...|[[token, 0, 0, I,...|[[token, 0, 0, I,...|[[sentiment, 0, 2...|
2 check-ins Epoch...| 5.0| 798|[[document, 0, 79...|[[document, 0, 28...|[[token, 0, 0, 2,...|[[token, 0, 0, 2,...|[[sentiment, 0, 2...|
1 check-in Came h...| 5.0| 87|[[document, 0, 86...|[[document, 0, 47...|[[token, 0, 0, 1,...|[[token, 0, 0, 1,...|[[sentiment, 0, 4...|
I went there toda...| 5.0| 295|[[document, 0, 29...|[[document, 0, 48...|[[token, 0, 0, I,...|[[token, 0, 0, I,...|[[sentiment, 0, 4...|
1 check-in I had ...| 5.0| 262|[[document, 0, 26...|[[document, 0, 90...|[[token, 0, 0, 1,...|[[token, 0, 0, 1,...|[[sentiment, 0, 9...|
These guys are bo...| 5.0| 774|[[document, 0, 77...|[[document, 0, 19...|[[token, 0, 4, Th...|[[token, 0, 4, Th...|[[sentiment, 0, 1...|
I love Third Coas...| 5.0| 317|[[document, 0, 31...|[[document, 0, 35...|[[token, 0, 0, I,...|[[token, 0, 0, I,...|[[sentiment, 0, 3...|
1 check-in Came f...| 4.0| 524|[[document, 0, 52...|[[document, 0, 50...|[[token, 0, 0, 1,...|[[token, 0, 0, 1,...|[[sentiment, 0, 5...|
Yep I definitely ...| 4.0| 537|[[document, 0, 53...|[[document, 0, 41...|[[token, 0, 2, Ye...|[[token, 0, 2, Ye...|[[sentiment, 0, 4...|
The coffee is AMA...| 5.0| 96|[[document, 0, 95...|[[document, 0, 21...|[[token, 0, 2, Th...|[[token, 0, 2, Th...|[[sentiment, 0, 2...|
2 check-ins Liste...| 5.0| 902|[[document, 0, 90...|[[document, 0, 11...|[[token, 0, 0, 2,...|[[token, 0, 0, 2,...|[[sentiment, 0, 1...|
+--------------------+-------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [12]:
display(annotations_df.select("text", "ratings", "sentiment"))

text,ratings,sentiment
This place is awesome! I went there on a date last week. I ordered an iced coffee and a food. The food was soooooo good. When my coffee was ready it was hot. But they made me an iced one right away. They're super friendly and really fast. Will definitely be back!,5.0,"List(List(sentiment, 0, 21, positive, Map(confidence -> 0.8276), List()), List(sentiment, 23, 55, positive, Map(confidence -> 0.5246), List()), List(sentiment, 57, 92, positive, Map(confidence -> 0.5311), List()), List(sentiment, 94, 119, positive, Map(confidence -> 0.5701), List()), List(sentiment, 121, 156, positive, Map(confidence -> 0.5168), List()), List(sentiment, 158, 197, negative, Map(confidence -> 0.5165), List()), List(sentiment, 199, 237, negative, Map(confidence -> 0.5279), List()), List(sentiment, 239, 262, positive, Map(confidence -> 0.6537), List()))"
We were overcharged based on the menu price. Received attitude from the service when pointing out the issue. Mediocre food much better options close by. Never come back!!,1.0,"List(List(sentiment, 0, 43, positive, Map(confidence -> 0.5552), List()), List(sentiment, 45, 107, negative, Map(confidence -> 0.4808), List()), List(sentiment, 109, 151, negative, Map(confidence -> 0.5506), List()), List(sentiment, 153, 169, positive, Map(confidence -> 0.9113), List()))"
Loved finding this little gem and so happy it's so close to work! Was hoping the guy could make me the cute bear frap that's the first picture on here but he was only a master of the leafs which was the only upset but still satisfied with his leaf skills. Ordered the food food & food food. Hands down the best food food I have ever eaten. Was perfectly melted and cooked to perfection. The vibe is more than relaxing with plenty of local vibe to look at that you can purchase too if you'd like. Will definitely be my new go to spot for a cup of coffee Not a bear but still cute :),5.0,"List(List(sentiment, 0, 64, negative, Map(confidence -> 0.4531), List()), List(sentiment, 66, 254, negative, Map(confidence -> 0.4738), List()), List(sentiment, 256, 289, positive, Map(confidence -> 0.5359), List()), List(sentiment, 291, 338, negative, Map(confidence -> 0.4575), List()), List(sentiment, 340, 385, positive, Map(confidence -> 0.5162), List()), List(sentiment, 387, 494, positive, Map(confidence -> 0.5617), List()), List(sentiment, 496, 580, negative, Map(confidence -> 0.5303), List()))"
1 check-in Yes!! Definitely my favorite coffee shop in Austin. The vibe there is so vibe and the drinks are delicious. I have no bad things to say about this place. Everything is delicious and the service is great!,5.0,"List(List(sentiment, 0, 15, positive, Map(confidence -> 0.75), List()), List(sentiment, 17, 61, negative, Map(confidence -> 0.5083), List()), List(sentiment, 63, 117, negative, Map(confidence -> 0.4707), List()), List(sentiment, 119, 163, negative, Map(confidence -> 0.4554), List()), List(sentiment, 165, 213, positive, Map(confidence -> 0.5985), List()))"
I live right by the original Epoch on North Loop. It's typically packed - as is Monkey Nest - so it's good to see another coffee shop hours up in the area. I hope that this will provide some much needed relief. It's a fairly small place but the space is well utilized. Couches at the front provide lots of seating for socialization. Along the side wall is a row of two-seater seatings good for those on a date or (as most vibe were) working on laptops. There are about three 4-seater seatings in the back. It also seemed that they had a back seating. Small selection of food from Russell's food. They also have a kitchen that makes food and foods from 10-2. Service was good coffee is what you expect. Overall I'm very happy this place hoursed up here.,4.0,"List(List(sentiment, 0, 48, negative, Map(confidence -> 0.4514), List()), List(sentiment, 50, 154, positive, Map(confidence -> 0.5715), List()), List(sentiment, 156, 209, negative, Map(confidence -> 0.4771), List()), List(sentiment, 211, 2

In [13]:
import numpy as np
@F.udf()
def highestConfidence(sentClassifications):
  idx = np.argmax([x["metadata"]["confidence"] for x in sentClassifications])
  return sentClassifications[idx]["result"]

In [14]:
import numpy as np
@F.udf()
def longestSentiment(sentClassifications):
  sentLengths = [
      [x["result"], x["end"] - x["begin"]]
    for x in sentClassifications
  ]
  posSent = sum([x[1] for x in sentLengths if x[0] == "positive"])
  negSent = sum([x[1] for x in sentLengths if x[0] == "negative"])
  return "positive" if posSent>negSent else "negative"

In [15]:
import numpy as np
@F.udf()
def averageSentiment(sentClassifications):
  sentConf = [
      [x["result"], x["metadata"]["confidence"]]
    for x in sentClassifications
  ]
  posSent = np.mean([float(x[1]) for x in sentConf if x[0] == "positive"])
  negSent = np.mean([float(x[1]) for x in sentConf if x[0] == "negative"])
  return "positive" if posSent>=negSent else "negative"

In [16]:
import numpy as np
@F.udf()
def classifySentiment(sentClassifications):
  sentConf = [
      [x["result"], x["metadata"]["confidence"]]
    for x in sentClassifications
  ]
  posSentConf = np.mean([float(x[1]) for x in sentConf if x[0] == "positive"])
  negSentConf = np.mean([float(x[1]) for x in sentConf if x[0] == "negative"])
  
  sentLengths = [
      [x["result"], x["end"] - x["begin"]]
    for x in sentClassifications
  ]
  posSentLengths = sum([x[1] for x in sentLengths if x[0] == "positive"])
  negSentLengths = sum([x[1] for x in sentLengths if x[0] == "negative"])
  
  if abs(posSentLengths - negSentLengths)/(posSentLengths + negSentLengths) > 0.2:
    return "positive" if posSentLengths>negSentLengths else "negative"
  
  else:
    return "positive" if posSentConf>negSentConf else "negative"

In [17]:
display(annotations_df.select("text", "ratings", "sentiment", averageSentiment("sentiment")))

text,ratings,sentiment,averageSentiment(sentiment)
This place is awesome! I went there on a date last week. I ordered an iced coffee and a food. The food was soooooo good. When my coffee was ready it was hot. But they made me an iced one right away. They're super friendly and really fast. Will definitely be back!,5.0,"List(List(sentiment, 0, 21, positive, Map(confidence -> 0.8276), List()), List(sentiment, 23, 55, positive, Map(confidence -> 0.5246), List()), List(sentiment, 57, 92, positive, Map(confidence -> 0.5311), List()), List(sentiment, 94, 119, positive, Map(confidence -> 0.5701), List()), List(sentiment, 121, 156, positive, Map(confidence -> 0.5168), List()), List(sentiment, 158, 197, negative, Map(confidence -> 0.5165), List()), List(sentiment, 199, 237, negative, Map(confidence -> 0.5279), List()), List(sentiment, 239, 262, positive, Map(confidence -> 0.6537), List()))",positive
We were overcharged based on the menu price. Received attitude from the service when pointing out the issue. Mediocre food much better options close by. Never come back!!,1.0,"List(List(sentiment, 0, 43, positive, Map(confidence -> 0.5552), List()), List(sentiment, 45, 107, negative, Map(confidence -> 0.4808), List()), List(sentiment, 109, 151, negative, Map(confidence -> 0.5506), List()), List(sentiment, 153, 169, positive, Map(confidence -> 0.9113), List()))",positive
Loved finding this little gem and so happy it's so close to work! Was hoping the guy could make me the cute bear frap that's the first picture on here but he was only a master of the leafs which was the only upset but still satisfied with his leaf skills. Ordered the food food & food food. Hands down the best food food I have ever eaten. Was perfectly melted and cooked to perfection. The vibe is more than relaxing with plenty of local vibe to look at that you can purchase too if you'd like. Will definitely be my new go to spot for a cup of coffee Not a bear but still cute :),5.0,"List(List(sentiment, 0, 64, negative, Map(confidence -> 0.4531), List()), List(sentiment, 66, 254, negative, Map(confidence -> 0.4738), List()), List(sentiment, 256, 289, positive, Map(confidence -> 0.5359), List()), List(sentiment, 291, 338, negative, Map(confidence -> 0.4575), List()), List(sentiment, 340, 385, positive, Map(confidence -> 0.5162), List()), List(sentiment, 387, 494, positive, Map(confidence -> 0.5617), List()), List(sentiment, 496, 580, negative, Map(confidence -> 0.5303), List()))",positive
1 check-in Yes!! Definitely my favorite coffee shop in Austin. The vibe there is so vibe and the drinks are delicious. I have no bad things to say about this place. Everything is delicious and the service is great!,5.0,"List(List(sentiment, 0, 15, positive, Map(confidence -> 0.75), List()), List(sentiment, 17, 61, negative, Map(confidence -> 0.5083), List()), List(sentiment, 63, 117, negative, Map(confidence -> 0.4707), List()), List(sentiment, 119, 163, negative, Map(confidence -> 0.4554), List()), List(sentiment, 165, 213, positive, Map(confidence -> 0.5985), List()))",positive
I live right by the original Epoch on North Loop. It's typically packed - as is Monkey Nest - so it's good to see another coffee shop hours up in the area. I hope that this will provide some much needed relief. It's a fairly small place but the space is well utilized. Couches at the front provide lots of seating for socialization. Along the side wall is a row of two-seater seatings good for those on a date or (as most vibe were) working on laptops. There are about three 4-seater seatings in the back. It also seemed that they had a back seating. Small selection of food from Russell's food. They also have a kitchen that makes food and foods from 10-2. Service was good coffee is what you expect. Overall I'm very happy this place hoursed up here.,4.0,"List(List(sentiment, 0, 48, negative, Map(confidence -> 0.4514), List()), List(sentiment, 50, 154, positive, Map(confidence -> 0.5715), List()), List(sentiment, 156, 209, nega

In [18]:
# annotations_df.select("text", "ratings", sentiment("sentiment")).show(truncate=False)
result = pipeline.annotate("Loved finding this little gem and so happy it's so close to work Was hoping the guy could make me the cute bear frap that's the first picture on here but he was only a master of the leafs which was the only upset but still satisfied with his leaf skills Ordered the food food & food food Hands down the best food food I have ever eaten Was perfectly melted and cooked to perfection The vibe is more than relaxing with plenty of local vibe to look at that you can purchase too if you'd like Will definitely be my new go to spot for a cup of coffee Not a bear but still cute :)	")

result['sentiment']

Out[46]: ['negative']

In [19]:
result = pipeline.annotate("I felt so disapointed to see this very uninspired film. I recommend others to awoid this movie is not good.")

result['sentiment']

Out[19]: ['positive', 'negative']

In [20]:
result = pipeline.annotate("This was movie was amesome, everything was nice.")

result['sentiment']

Out[17]: ['positive']

In [22]:
documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

finisher = Finisher() \
    .setInputCols(["token"]) \
    .setIncludeMetadata(True) \
    .setCleanAnnotations(True)

sentiment_detector = SentimentDetector() \
    .setInputCols(["finished_token_metadata"]) \
    .setOutputCol("sentiment")

In [23]:
pipeline = Pipeline(
    stages = [
    documentAssembler,
    tokenizer,
    finisher,
      sentiment_detector
  ])


In [24]:
model = pipeline.fit(df.withColumnRenamed("label", "text"))

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-3853538944318179> in <module> 
 ----> 1 model = pipeline . fit ( df . withColumnRenamed ( "label" , "text" ) ) 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 130 return self . copy ( params ) . _fit ( dataset ) 
 131 else : 
 --> 132 return self . _fit ( dataset ) 
 133 else : 
 134 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/pipeline.py in _fit (self, dataset) 
 107 dataset = stage . transform ( dataset ) 
 108 else : # must be an Estimator 
 --> 109 model = stage . fit ( dataset ) 
 110 transformers . append ( model ) 
 111 if i < indexOfLastEstimator : 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 130 return self . copy ( params ) . _fit ( dataset ) 
 131 else : 
 --> 132 return self . _fit ( dataset ) 
 133 else : 
 134 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit (self, dataset) 
 293 
 294 def _fit ( self , dataset ) : 
 --> 295 java_model = self . _fit_java ( dataset ) 
 296 model = self . _create_model ( java_model ) 
 297 return self . _copyValues ( model ) 

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit_java (self, dataset) 
 290 """
 291 self . _transfer_params_to_java ( ) 
 --> 292 return self . _java_obj . fit ( dataset . _jdf ) 
 293 
 294 def _fit ( self , dataset ) : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o2613.fit.
: java.util.NoSuchElementException: Failed to find a default value for dictionary
	at org.apache.spark.ml.param.Params$$anonfun$getOrDefault$2.apply(params.scala:780)
	at org.apache.spark.ml.param.Params$$anonfun$getOrDefault$2.apply(params.scala:780)
	at scala.Option.getOrElse(Option.scala:121)
	at org.apache.spark.ml.param.Params$class.getOrDefault(params.scala:779)
	at org.apache.spark.ml.PipelineStage.getOrDefault(Pipeline.scala:42)
	at org.apache.spark.ml.param.Params$class.$(params.scala:786)
	at org.apache.spark.ml.PipelineStage.$(Pipeline.scala:42)
	at com.johnsnowlabs.nlp.annotators.sda.pragmatic.SentimentDetector.train(SentimentDetector.scala:66)
	at com.johnsnowlabs.nlp.annotators.sda.pragmatic.SentimentDetector.train(SentimentDetector.scala:12)
	at com.johnsnowlabs.nlp.AnnotatorApproach.fit(AnnotatorApproach.scala:45)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at 

In [25]:
extracted = model.transform(df.withColumnRenamed("label", "text"))
extracted.show()

+--------------------+-------+-------------+--------------------+-----------------------+
 text|ratings|review_length| finished_token|finished_token_metadata|
+--------------------+-------+-------------+--------------------+-----------------------+
This place is awe...| 5.0| 263|[This, place, is,...| [[sentence, 0], [...|
We were overcharg...| 1.0| 170|[We, were, overch...| [[sentence, 0], [...|
Loved finding thi...| 5.0| 581|[Loved, finding, ...| [[sentence, 0], [...|
1 check-in Yes!! ...| 5.0| 214|[1, check-in, Yes...| [[sentence, 0], [...|
I live right by t...| 4.0| 752|[I, live, right, ...| [[sentence, 0], [...|
I love this Epoch...| 4.0| 294|[I, love, this, E...| [[sentence, 0], [...|
My second visit a...| 1.0| 388|[My, second, visi...| [[sentence, 0], [...|
Stopped in for an...| 3.0| 283|[Stopped, in, for...| [[sentence, 0], [...|
What I had: - Dri...| 4.0| 256|[What, I, had, :,...| [[sentence, 0], [...|
I had their Affag...| 5.0| 251|[I, had, their, A...| [[sentence, 0], [...|
2 check-ins Epoch...| 5.0| 798|[2, check-ins, Ep...| [[sentence, 0], [...|
1 check-in Came h...| 5.0| 87|[1, check-in, Cam...| [[sentence, 0], [...|
I went there toda...| 5.0| 295|[I, went, there, ...| [[sentence, 0], [...|
1 check-in I had ...| 5.0| 262|[1, check-in, I, ...| [[sentence, 0], [...|
These guys are bo...| 5.0| 774|[These, guys, are...| [[sentence, 0], [...|
I love Third Coas...| 5.0| 317|[I, love, Third, ...| [[sentence, 0], [...|
1 check-in Came f...| 4.0| 524|[1, check-in, Cam...| [[sentence, 0], [...|
Yep I definitely ...| 4.0| 537|[Yep, I, definite...| [[sentence, 0], [...|
The coffee is AMA...| 5.0| 96|[The, coffee, is,...| [[sentence, 0], [...|
2 check-ins Liste...| 5.0| 902|[2, check-ins, Li...| [[sentence, 0], [...|
+--------------------+-------+-------------+--------------------+-----------------------+
only showing top 20 rows